In [4]:
import sys; sys.path.append('../shared_functions')

In [13]:
import random
from time import time
from datetime import datetime, timedelta
from function_a import AggModelA, RawModel

In [14]:
val, dt = random.random(), int(time())
load = RawModel(value=val, date=dt)

In [50]:
from function_a import print_windowed_events, source

In [79]:
from function_a import app

In [86]:
from datetime import datetime, timedelta
from time import time
import faust
import logging
import os
import pytest
import random
import sys

if os.environ.get('DEV') is None:
    sys.path.append('../data_processing')

from processing_module.app import app
logger = logging.getLogger(__name__)

TOPIC = 'raw-event'
SINK = 'agg-event-a'
TABLE = 'table_b'
CLEANUP_INTERVAL = 7.5
WINDOW = 5
WINDOW_EXPIRES = 10
PARTITIONS = 1


class RawModel(faust.Record):
    date: datetime
    value: float


class AggModelA(faust.Record):
    date: datetime
    count: int
    mean: float


app.conf.table_cleanup_interval = CLEANUP_INTERVAL
source = app.topic(TOPIC, partitions=PARTITIONS, value_type=RawModel)
sink = app.topic(SINK, value_type=AggModelA)


def window_processor(key, events):
    timestamp = key[1][0]
    values = [event.value for event in events]
    count = len(values)
    mean = sum(values) / count

    logger.info(f"""
    processing window:
    {len(values)} events,
    mean: {mean:.2f},
    timestamp {timestamp}
    """)

    sink.send_soon(value=AggModelA(date=timestamp, count=count, mean=mean))


tumbling_table = (app.Table(
    TABLE,
    default=list,
    partitions=PARTITIONS,
    on_window_close=window_processor,
).tumbling(WINDOW,
           expires=timedelta(seconds=WINDOW_EXPIRES)).relative_to_field(
               RawModel.date))


@app.agent(source)
async def print_windowed_events(stream):
    async for event in stream:
        value_list = tumbling_table['events'].value()
        value_list.append(event)
        tumbling_table['events'] = value_list
        yield event


@app.timer(0.1)
async def produce():
    await source.send(value=RawModel(value=random.random(), date=int(time())))

In [96]:
@no_type_check
main()

SyntaxError: invalid syntax (<ipython-input-96-d81155b64f9c>, line 2)

In [63]:
@app.task
async def on_startup(app):
    print('STARTING UP: %r' % (app,))

In [65]:
app.LiveCheck().

False

In [66]:
import faust
import pytest

@pytest.mark.asyncio()
async def test_process_order():
    app.conf.store = 'memory://'
    async with process_order.test_context() as agent:
        order = Order(account_id='1', product_id='2', amount=1, price=300)
        event = await agent.put(order)

        # windowed table: we select window relative to the current event
        assert orders_for_account['1'].current(event) == 1

        # in the window 3 hours ago there were no orders:
        assert orders_for_account['1'].delta(3600 * 3, event)


class Order(faust.Record, serializer='json'):
    account_id: str
    product_id: str
    amount: int
    price: float

app = faust.App('test-example')
orders_topic = app.topic('orders', value_type=Order)

# order count within the last hour (window is a 1-hour TumblingWindow).
orders_for_account = app.Table(
    'order-count-by-account', default=int,
).tumbling(3600).relative_to_stream()

@app.agent(orders_topic)
async def process_order(orders):
    async for order in orders.group_by(Order.account_id):
        orders_for_account[order.account_id] += 1
        yield order

In [78]:
def main() -> None:
    app.main()

In [38]:
window_processor(print_windowed_events())

TypeError: window_processor() missing 1 required positional argument: 'events'